In [3]:
%pip install scikit-learn
%pip install numpy
%pip install pandas
%pip install seaborn

/nix/store/mp4nbfmc0a42shg6w2vam5d9vzby4a4k-python3-3.10.11-env/bin/python3.10: No module named pip
Note: you may need to restart the kernel to use updated packages.
